*This notebook contains an excerpt from the [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) by Jake VanderPlas; the content is available [on GitHub](https://github.com/jakevdp/PythonDataScienceHandbook).*

# Hierarchical Indexing

In [8]:
import pandas as pd
import numpy as np

In [9]:
population_data = [{'state': 'California', 'year': '2000', 'population': 33871648},
 {'state': 'California', 'year': '2010', 'population': 37253956},
 {'state': 'New York', 'year': '2000', 'population': 18976457},
 {'state': 'New York', 'year': '2010', 'population': 19378102},
 {'state': 'Texas', 'year': '2000', 'population': 20851820},
 {'state': 'Texas', 'year': '2010', 'population': 25145561}]

In [10]:
pop_df = pd.DataFrame(population_data)
pop_df

,population,state,year
0,33871648,California,2000
1,37253956,California,2010
2,18976457,New York,2000
3,19378102,New York,2010
4,20851820,Texas,2000
5,25145561,Texas,2010


Select records of California

In [4]:
pop_df[pop_df['state']=='California']

,population,state,year
0,33871648,California,2000
1,37253956,California,2010


In [12]:
pop_df2 = pop_df.set_index('state')
pop_df2

,population,year
state,,
California,33871648,2000
California,37253956,2010
New York,18976457,2000
New York,19378102,2010
Texas,20851820,2000
Texas,25145561,2010


In [13]:
pop_df2.loc[['California', 'Texas']]

,population,year
state,,
California,33871648,2000
California,37253956,2010
Texas,20851820,2000
Texas,25145561,2010


In [7]:
pop_df2.reset_index()

,state,population,year
0,California,33871648,2000
1,California,37253956,2010
2,New York,18976457,2000
3,New York,19378102,2010
4,Texas,20851820,2000
5,Texas,25145561,2010


What is the population of California in year 2000?

In [14]:
pop_df

,population,state,year
0,33871648,California,2000
1,37253956,California,2010
2,18976457,New York,2000
3,19378102,New York,2010
4,20851820,Texas,2000
5,25145561,Texas,2010


In [16]:
#(pop_df['year']=='2000') & (pop_df['state']=='California')

In [8]:
q = pop_df[(pop_df['year']=='2000') & (pop_df['state']=='California')]
q

,population,state,year
0,33871648,California,2000


In [9]:
q.loc[0, 'population']

33871648

In [17]:
pop_df2.loc['California']

,population,year
state,,
California,33871648,2000
California,37253956,2010


In [19]:
pop_df2[pop_df2['year']=='2000'].loc['California', 'population']

33871648

In [21]:
#pop_df2.loc[('California', '2000')]

In [23]:
pop_df3 = pop_df.set_index(['state', 'year'])
pop_df3

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

In [25]:
pop_df3.index

MultiIndex(levels=[['California', 'New York', 'Texas'], ['2000', '2010']],
           codes=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]],
           names=['state', 'year'])

In [26]:
pop_df3.loc['California']

,population
year,
2000,33871648
2010,37253956


In [27]:
pop_df3.loc[('California', '2010')]

population    37253956
Name: (California, 2010), dtype: int64

In [31]:
pop_df4 = pop_df3.reset_index().set_index(['year', 'state'])
pop_df4

,,population
year,state,
2000,California,33871648
2010,California,37253956
2000,New York,18976457
2010,New York,19378102
2000,Texas,20851820
2010,Texas,25145561


In [32]:
pop_df4.loc['2010']

,population
state,
California,37253956
New York,19378102
Texas,25145561


In [33]:
pop_df3

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

In [17]:
pop_df3.loc[(:, '2010')]

SyntaxError: invalid syntax (<ipython-input-17-cf78904ad9b7>, line 1)

In [18]:
pop_df3

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

In [36]:
idx = pd.IndexSlice
pop_df3.loc[idx[['California', 'Texas'], '2010'], 'population']

state       year
California  2010    37253956
Texas       2010    25145561
Name: population, dtype: int64

In [38]:
s = pop_df3['population']
s

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
Name: population, dtype: int64

In [43]:
s.mean()

25912924.0

In [46]:
#pop_df.mean()

In [47]:
s.unstack('year').mean()

year
2000    2.456664e+07
2010    2.725921e+07
dtype: float64

In [48]:
s.unstack('state').mean()

state
California    35562802.0
New York      19177279.5
Texas         22998690.5
dtype: float64

### MultiIndex as extra dimension

You might notice something else here: we could easily have stored the same data using a simple ``DataFrame`` with index and column labels.
In fact, Pandas is built with this equivalence in mind. The ``unstack()`` method will quickly convert a multiply indexed ``Series`` into a conventionally indexed ``DataFrame``:

In [20]:
pop_df3['under18'] = [9267089, 9284094,
                      4687374, 4318033,
                      5906301, 6879014]
pop_df3

population  under18
state      year                     
California 2000    33871648  9267089
           2010    37253956  9284094
New York   2000    18976457  4687374
           2010    19378102  4318033
Texas      2000    20851820  5906301
           2010    25145561  6879014

In [21]:
f_u18 = pop_df3['under18'] / pop_df3['population']
f_u18

state       year
California  2000    0.273594
            2010    0.249211
New York    2000    0.247010
            2010    0.222831
Texas       2000    0.283251
            2010    0.273568
dtype: float64

In [22]:
f_u18.unstack()

year,2000,2010
state,,
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


In [23]:
f_u18.unstack('state')

state,California,New York,Texas
year,,,
2000,0.273594,0.247010,0.283251
2010,0.249211,0.222831,0.273568


In [24]:
f_u18.unstack('state').stack()

year  state     
2000  California    0.273594
      New York      0.247010
      Texas         0.283251
2010  California    0.249211
      New York      0.222831
      Texas         0.273568
dtype: float64

# Using multi-index to calculate aggregations

In [25]:
f_u18

state       year
California  2000    0.273594
            2010    0.249211
New York    2000    0.247010
            2010    0.222831
Texas       2000    0.283251
            2010    0.273568
dtype: float64

In [26]:
f_u18.mean()

0.25824409535188225

In [27]:
f_u18.unstack('state')

state,California,New York,Texas
year,,,
2000,0.273594,0.247010,0.283251
2010,0.249211,0.222831,0.273568


In [28]:
f_u18.unstack('state').mean()

state
California    0.261403
New York      0.234920
Texas         0.278409
dtype: float64

In [29]:
f_u18.unstack('year').mean()

year
2000    0.267952
2010    0.248536
dtype: float64